In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import gc
import random
import re
from sklearn.metrics import *
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
base_path = '../preprocessed data/'

train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
train_quality = pd.read_csv(base_path + 'new_train_quality.csv')
train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])

# feature engineering

case3. model_nm + fwver + errtype + counts + multiple_model + errtype_num + period

In [4]:
df1 = train_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0


In [5]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,1
...,...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0,1
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0,1


In [6]:
# multiple_model
model_df =  train_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
10000,0
10001,0
10002,0
10003,0
10004,0
10005,0
10006,1


In [7]:
# errtype_num
errtype_df = train_err[['user_id', 'errtype']]
errtype_df = errtype_df.user_id.value_counts(sort=False).to_frame(name='errtype_num')
errtype_df.head()

,errtype_num
10000,316
10001,2345
10002,305
10003,297
10004,777


In [8]:
# period
train_err['day'] = train_err.time.dt.day
train_err['hour'] = train_err.time.dt.hour

user_day_df = train_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df['period'] = 0 

for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

user_day_df

,user_id,day,period
0,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
1,10001,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
2,10002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
3,10003,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
4,10004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]",10
14996,24996,[30],1
14997,24997,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14998,24998,"[1, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19...",21


In [9]:
user_day_df.index = user_day_df['user_id'] - 10000
user_day_df.drop(columns=['user_id', 'day'], inplace=True)

user_day_df

,period
user_id,
0,30
1,30
2,29
3,30
4,30
...,...
14995,10
14996,1
14997,30


In [10]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367996,24999,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15367997,24999,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15367998,24999,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15367999,24999,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# 모든 column 합쳐주기
df1 = df1.groupby('user_id').sum()
dfs = [df1, user_has_multiple_models, errtype_df, user_day_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_model,errtype_num,period
user_id,,,,,,,,,,,,,,,,,,,,,
10000,316.0,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,30.0
10001,2345.0,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,2345.0,30.0
10002,305.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,305.0,25.0
10003,297.0,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,297.0,30.0
10004,777.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.0,777.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,191.0,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0,191.0,NaN
24996,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN
24997,818.0,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,58.0,7.0,5.0,0.0,818.0,NaN


In [13]:
# 중복정보가 있는 column 제거
def remove_collinearity(X, threshold):
    """
    X : feature matrix
    threshold : 다중공선성을 제거할 column을 고르는 기준 값. [0, 1]
    """
    
    corr = X.corr()
    candidate_cols = []
    
    for x in corr.iterrows():
        idx, row = x[0], x[1] # decoupling tuple
        if idx in candidate_cols:
            continue
        candidates = row[row > threshold].index[1:]

        # 자기 자신을 제외하고 threshold를 넘는 column이 있다면 제거 할 column이다.
        if len(candidates) != 0:
            for col in candidates:
                candidate_cols.append(col)           
    
    return candidate_cols

In [17]:
correlated_features = remove_collinearity(dfs, 0.8)
print(correlated_features)
train_8 = dfs.drop(columns=correlated_features)
len(train_8.columns)

['errtype_num', 'fwver_04.22.1750', 'fwver_04.16.3553', 'fwver_04.33.1185', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_04.82.1778', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_7', 'errtype_11', 'errtype_12', 'errtype_16', 'errtype_21', 'errtype_23', 'errtype_37']


69

In [18]:
correlated_features = remove_collinearity(dfs, 0.9)
print(correlated_features)
train_9 = dfs.drop(columns=correlated_features)
len(train_9.columns)

['errtype_num', 'fwver_04.22.1750', 'fwver_04.16.3553', 'fwver_04.33.1261', 'fwver_05.15.2138', 'errtype_4', 'fwver_03.11.1167', 'errtype_27', 'errtype_28', 'fwver_04.82.1684', 'fwver_10', 'fwver_05.66.3237', 'fwver_04.73.2237', 'errtype_7', 'errtype_11', 'errtype_12', 'errtype_16', 'errtype_21', 'errtype_23', 'errtype_37']


71

In [20]:
train_8.index = train_8.index-10000
train_9.index = train_9.index-10000

In [22]:
train_8.to_csv('../training set/train_case3_8.csv', index=False)
train_9.to_csv('../training set/train_case3_9.csv', index=False)